# <center> **ПРОЕКТ "Решение комплексной бизнес-задачи с помощью SQL и Python"**

### ЦЕЛЬ
Подготовить основу рекомендательной системы, благодаря которой можно будет предлагать клиентам интересные им курсы.
### ЗАДАЧИ
1.	Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL.
2.	Обработать данные средствами Python.
3.	Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчёт продакт-менеджеру.


### Анализ данныx с помощью SQL 
Для изучения датасета, который должен стать основой для будущей рекомендательной системы, зададим себе некоторые вопросы и найдем на них ответы с помощью следующих кодов SQL.

##### Встречаются ли в одной пользовательской корзине разные типы покупок?
SELECT
cart_id,
COUNT(distinct resource_type)
FROM final.cart_items 
group by cart_id
having COUNT(distinct resource_type) > 1

Полученный результат показал, что в рамках одной корзины встречаются покупки разных типов (Course и Product)


##### Встречаются ли пользовательские корзины, которые принадлежат разным покупателям?
SELECT
id,
COUNT(distinct user_id)
FROM final.carts 
group by id
having COUNT(distinct user_id) > 1

Полученный результат показал, что одна корзина может принадлежать только одному пользователю


##### Встречаются ли пользователи, которым принадлежат разные корзины?
SELECT
user_id,
COUNT(distinct id)
FROM final.carts 
group by user_id
having COUNT(distinct id) > 1

Полученный результат показал, что одному покупателю может принадлежать несколько корзин


##### Встречаются ли в датасете незавершенные покупки курсов, т.е. такие строки с записями курсов, у которых отсутствует дата оплаты?
SELECT *
FROM final.carts as cs
JOIN final.cart_items as ci on cs.ID = ci.cart_ID
WHERE resource_type = 'Course' and purchased_at is null

Полученный результат показал, что датасет содержит незавершенные покупки курсов, содержит строки с записями курсов, у которых отсутствует дата оплаты

##### Результаты, полученные выше, необходимо учесть при написании SQL-запроса к базе данных

In [1]:
import psycopg2
import psycopg2.extras 
import pandas as pd
from collections import deque
from collections import defaultdict
from collections import Counter

### Сформируем SQL-запрос для получения необходимых сведений о продажах учебных курсов на основе предложенного датасета.

Для решения поставленной задачи достаточно данных, содержащихся в полях user_id (id пользователя) и resource_id (id курса) таблиц carts (с данными о пользовательских корзинах) и cart_items (с данными о коммерческих продуктах, добавленных пользователями в корзину), таблиц, объединенных по идентификаторам пользовательских корзин. При этом полученные с помощью SQL-запроса данные должны содержать записи только приобретенных курсов, т.е. для отбора записей в таблицу, будущую основой для нашего исследования, запрос должен включать два следующих условия: 1. наличие даты покупки (поле purchased_at не пустое), 2. тип приобретенного продукта (resource_type) - курс (Course), т.к. по условиям поставленной задачи основой рекомендательной системы должны стать сведения за прошлые периоды именно о ПРОДАЖАХ КУРСОВ. Кроме того, изучение датасета показало, что один и тот же пользователь мог купить один и тот же курс не один раз (в рамках разных пользовательских корзин), а это означает, что для того, чтобы полученная нами таблица не содержала дублей user_id - resource_id, нам необходимо объединить такие возможные дублирующиеся строки (что также логично соответствует поставленной задаче, т.к. собирать курсы в пары необходимо в рамках покупок отдельных пользователей). Этого можно достичь при помощи группировки строк SQL-запросом по двум указанным признакам (user_id, resource_id). Также такую задачу можно решить с помощью метода drop_duplicates, применив его к полученной в результате отработанного SQL-запроса таблице. Но мы используем группировку в SQL-запросе.

In [2]:
def getCourseCountOnUser():
    query = '''SELECT user_id, resource_id
FROM final.carts as cs
JOIN final.cart_items as ci on cs.ID = ci.cart_ID
WHERE purchased_at is not null and resource_type = 'Course'
group by 1,2
order by 1
    '''.format()
    try:
        conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
        dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        dict_cur.execute(query)
        rows = dict_cur.fetchall()
        data = []
        for row in rows:
            data.append(dict(row))
        return data
    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if conn:
            dict_cur.close()
            conn.close()

Purch_Courses_On_User_df = pd.DataFrame(getCourseCountOnUser())
Purch_Courses_On_User_df

,user_id,resource_id
0,51,516
1,51,1099
2,974,356
3,2233,1125
4,2581,1187
...,...,...
70419,2190509,742
70420,2190575,757
70421,2190589,1125
70422,2190991,489


In [3]:
Purch_Courses_On_User_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70424 entries, 0 to 70423
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      70424 non-null  int64
 1   resource_id  70424 non-null  int64
dtypes: int64(2)
memory usage: 1.1 MB


### Определим правила построения рекомендательной системы.

ПЕРВАЯ РЕКОМЕНДАЦИЯ, которая будет даваться слушателям курсов при оформлении ими корзины покупок, будет формироваться следующим образом. Определим курс, который на основании полученных данных чаще всего покупался с курсом, для которого формируется рекомендация, и включим его в таблицу рекомендаций под № 1. При этом, как можно догадаться, существует вероятность того, что такого курса может не оказаться, т.е. с курсом, к которому подбирается рекомендация, другие курсы пользователями не покупались, или покупались, но слишком редко для определения одного из них, как рекомендуемого. Границей "слишком редко", т.е. минимально допустимой частотой повторений для включения в рекомендательную систему какой-либо пары курсов, встречающейся в покупках разных пользователей, будем считать число, равное 5-ти %%-там от числа повторений самой часто встречающейся в покупках разных пользователей пары курсов. Т.е. пары, частотность которых в покупках пользователей менее установленной границы, будут исключены рекомендательной системой автоматически.

При этом в случае, если мы будем иметь дело с курсом, для которого не найдется рекомендуемого курса исходя из частоты его повторений в парах с другими курсами, тогда в качестве рекомендации при покупке пользователем такого курса будет предлагаться курс, ставший лидером продаж в рассматриваемом периоде.

ВТОРАЯ РЕКОМЕНДАЦИЯ, которая будет даваться слушателям курсов при оформлении ими корзины покупок, будет формироваться схожим с формированием рекомендации № 1 образом, но с некоторыми отличиями. Курс-претендент на роль рекомендации № 2 должен соответствовать двум следующим критериям. В покупках разных пользователей такой курс-претендент должен быть вторым по частотности его приобретения в паре с курсом, для которого формируется рекомендация. При этом такая частотность не должа быть ниже установленной границы минимально допустимой частоты приобретения пользователями курса-претендента в паре с курсом, для которого формируется рекомендация. В данном случае такую границу установим на уровне 10-ти %%-в от числа повторений самой часто встречающейся в покупках разных пользователей пары курсов.

В случае, если таким критериям не будет соотвествовать ни один курс-претендент на роль рекомендации № 2 исходя из частоты его повторений в парах с другими курсами, тогда в качестве рекомендации при покупке пользователем такого курса будет предлагаться одно из двух: или курс, ставший лидером продаж в рассматриваемом периоде, или курс, занимающий в рассматриваемом периоде второе место по продажам после лидера, если курс-лидер стал рекомендацией № 1.

КОММЕНТАРИЙ: Следует пояснить, почему в случае со второй рекомендацией принята более высокая граница отсечения курсов, нерекомендуемых к покупке исходя из частоты их повторения в парах с другими курсами (10%). 
Мы видим, что в основу рекомендательной системы заложены два принципа: частотность воспроизводства различных пар различными пользователями (т.о. мы прогнозируем пользовательскую заинтересованность в курсах исходя из их текущего выбора) и общая популярность, востребованность отдельных курсов среди аудитории ресурса (т.о. мы прогнозируем пользовательскую заинтересованность в курсах исходя из общей статистики, из условно общепользовательских предпочтений). Как несложно догадаться, в первом случае мы опираемся на личные предпочтения пользователя, во втором - на консолидированные предпочтения достаточно большой выборки пользователей. При этом мы ожидаем, что личные предпочтения пользователя обеспечат более точный прогноз, чем условно общие. Но, если иметь ввиду границу отсечения полученных результатов статистики, как незаслуживающих внимания, на уровне 5-ти %-в, то мы понимаем, что подтверждение личных предпочтений лишь 5-тью %%-ми от возможных положительных статистических результатов и использование таких данных для прогноза пользовательских предпочтений, скорее всего, будет давать большую погрешность в прогнозе. Поэтому, предпринимая некоторую попытку компенсации такой возможной погрешности, в случае формирования второй рекомендации мы повышаем границу отсечения до 10-ти %-в, по-прежнему, хоть и в меньшей степени, оставляя место для проявления личных предпочтений при формировании рекомендаций (при этом также имея ввиду, что, если и стоило больше "доверять" личным предпочтениям, то при формировании первой рекомендации такой приоритет был отдан именно им, и шанс опереться на личные предпочтения упущен не был). Кроме того, мы также понимаем, что увеличение границы отсечения до 10-ти %-в, хоть и призвано повысить вероятность "попадания" прогнозов в предпочтения пользователей, совершающих покупку, однако такое увеличение также может давать невысокий уровень достоверности прогнозов. Но выбирая между рекомендациями, основанными на личных предпочтениях (с заданными границами отсечения), и рекомендациями, основанными на предпочтениях условно общепользовательских, все же оставим чуть больше места рекомендациям, основанным на личных предпочтениях, принимая во внимание то, что имеющиеся в нашем распоряжении данные, содержат не так много статистики, позволяющей нам делать некоторые прогнозы относительно личных предпочтений пользователей, что вынуждает нас т.о. при формировании рекомендательной системы в основном опираться на условно общепользовательские предпочтения. Также не будем забывать и о том, что заложенные в данную модель %%-ты отсечения доступны для коррекции, чем в дальнейшем при необходимости можно будет воспользоваться с учетом полученных практических результатов.

### Решение задачи

Из всего сказанного выше следует, что для решения поставленной задачи необходимо совершить следующие шаги:
1. получить список пользователей, совершивших за 2017-2018 гг. покупку более одного курса (необходимо для формирования рекомендаций на основании частоты повторения отдельных курсов в парах с другими курсами);
2. получить наборы курсов, проданных каждому пользователю, купившему более одного курса (из списка полученного на предыдущем шаге);
3. получить все пары курсов из покупок пользователей, купивших более одного курса, и составить из них два списка: оба списка будут состоять из одних и тех же пар, но последовательность записи курсов в каждой паре второго списка будет изменена на противоположную по отношению к записи курсов в парах первого списка, т.е. те курсы, которые в парах первого списка были первыми, в парах второго списка станут вторыми (так любой курс из любой пары станет в равной степени доступным при формировании рекомендаций);
4. рассчитать число повторений различных пар курсов в покупках пользователей для выявления наиболее часто встречающихся;
5. рассчитать две границы отсечения курсов (5%, 10%), нерекомендуемых к покупке исходя из частоты их повторения в парах с другими курсами;
6. на основании рассчитанной частоты повторения курсов в парах с другими курсами, а также с учетом границ "отсечения" составить промежуточную (исходя из частоты повторения курсов в парах с другими курсами) таблицу рекомендаций (№№ 1,2) для приобретаемых курсов;
7. определить два самых востребованных курса для формирования рекомендаций в случаях, когда курс в парах с другими курсами пользователями не покупался или покупался слишком редко;
8. составить полный список курсов, для которых строится система рекомендаций;
9. составить итоговую таблицу рекомендаций.

In [4]:
# ШАГ 1

# Получим список id пользователей, купивших более одного курса (на основании таблицы
# Purch_Courses_On_User_df, полученной при обращении SQL-запроса к базе данных и
# содержащей сведения о продажах курсов за 2017-2018 гг.)
user_id = Purch_Courses_On_User_df['user_id'].value_counts()
user_id = pd.DataFrame(user_id)
user_id_for_pairs_df = user_id[user_id['count'] > 1]
user_id_for_pairs_list = list(user_id_for_pairs_df.index)
print(len(user_id_for_pairs_list),type(user_id_for_pairs_list), user_id_for_pairs_list)

12656 <class 'list'> [694146, 1050532, 953401, 722252, 992470, 109035, 698038, 565025, 1017561, 190140, 713036, 1139935, 706819, 1505409, 1059131, 103634, 698699, 103734, 619070, 706876, 707274, 191574, 170284, 708012, 970344, 727622, 188586, 1098269, 463365, 176376, 211143, 206025, 189975, 1094306, 105574, 602903, 107708, 175538, 1033627, 643373, 724001, 1321546, 703769, 715142, 966156, 1151774, 910711, 1029696, 195166, 425067, 1112333, 750206, 897889, 305113, 1238463, 930808, 1843278, 273199, 644052, 695485, 164998, 750856, 207035, 740811, 104130, 1425060, 702745, 718271, 729637, 706482, 107135, 194848, 366779, 1149651, 1859358, 187265, 1140962, 1836516, 938359, 189219, 746252, 1091462, 729493, 724617, 1026652, 1207923, 749920, 192366, 1576797, 1618069, 186168, 1641925, 185996, 731123, 1009210, 719878, 1092994, 1098779, 1152982, 224507, 104460, 715980, 161848, 696042, 1868646, 747823, 691049, 1923213, 731985, 1106420, 692648, 1623955, 175686, 716796, 1029903, 689753, 1146579, 231758,

In [5]:
# ШАГ 2

# Получим список всех покупок в формате пар: id пользователя - id приобретенного курса
dq = deque(Purch_Courses_On_User_df.user_id)
dq2 = deque(Purch_Courses_On_User_df.resource_id)
len_df = len(Purch_Courses_On_User_df)
Purch_Courses_On_User_list = []
for i in range(len_df):
    one_purch_user_list = []
    rec1 = dq.pop()
    rec2 = dq2.pop()
    one_purch_user_list.append(rec1)
    one_purch_user_list.append(rec2)
    Purch_Courses_On_User_list.append(one_purch_user_list)

print(len(Purch_Courses_On_User_list),Purch_Courses_On_User_list)


# Соберем все приобретенные курсы в группы (наборы) по пользователям (пока не 
# исключая тех пользователей, которые приобрели только один курс)
Purch_Courses_On_User_ddict = defaultdict(set)
for user_id, resource_id in Purch_Courses_On_User_list:
    Purch_Courses_On_User_ddict[user_id].add(resource_id)
 
print(len(Purch_Courses_On_User_ddict),Purch_Courses_On_User_ddict)


# Получим наборы курсов, проданных пользователям, купившим более одного курса
# (каждый набор будет соответствовать набору курсов, приобретенных каждым 
# пользователем, но эти наборы в результате сделаем обезличенным, т.е. сведения
# об id пользователей, приобретавщих курсы, из итогового списка исключим)
lists_courses_on_users = []
for i in user_id_for_pairs_list:
    lists_courses_on_users.append(Purch_Courses_On_User_ddict[i])
print(len(lists_courses_on_users), lists_courses_on_users)

70424 [[2191100, 569], [2190991, 489], [2190589, 1125], [2190575, 757], [2190509, 742], [2190496, 515], [2190458, 570], [2190141, 1185], [2190141, 794], [2190141, 756], [2189968, 1104], [2189962, 794], [2189912, 504], [2189877, 566], [2189622, 566], [2189609, 1186], [2189551, 553], [2189538, 679], [2189494, 566], [2189456, 791], [2189451, 794], [2189446, 490], [2189363, 794], [2189319, 794], [2189302, 507], [2189170, 791], [2189102, 840], [2188990, 750], [2188926, 743], [2188926, 515], [2188868, 502], [2188790, 566], [2188677, 566], [2188575, 1152], [2188289, 490], [2188216, 1185], [2188154, 1125], [2188112, 794], [2188103, 566], [2188057, 742], [2187821, 1186], [2187820, 564], [2187809, 840], [2187753, 1186], [2187691, 1152], [2187672, 909], [2187625, 356], [2187601, 912], [2187601, 765], [2187601, 571], [2187601, 553], [2187601, 356], [2187441, 752], [2187424, 749], [2187376, 912], [2187244, 1125], [2187231, 569], [2187184, 1152], [2187055, 1125], [2186903, 908], [2186581, 1129], [21

In [6]:
# ШАГ 3

# Получим все пары курсов, сложившиеся в покупках пользователей, купивших более 
# 1-го курса, и запишем их в два аналогичных списка таким образом, чтобы при 
# чтении записей id курсов в парах первого списка слева направо пары первого 
# списка оказались бы идентичными парам курсов второго списка при чтении записей
# id курсов во вторых справа налево. В дальнейшем эти списки понадобятся для
# расчета частотности указанных пар в покупках с использованием объекта Counter.
pairs_list = []
pairs_dupl_list = []
for elem in lists_courses_on_users:
    elem = sorted(list(elem))
    k = 0
    for i in range(len(elem)-1):
        k += 1
        for j in range(len(elem)-k):
            pair_list = (str(elem[i]) + str(elem[j+k]), len(str(elem[i])))
            pair_dupl_list = (str(elem[j+k]) + str(elem[i]), len(str(elem[j+k])))

            pairs_list.append(pair_list)
            pairs_dupl_list.append(pair_dupl_list)

print(len(pairs_list))
print(pairs_list)
print(len(pairs_dupl_list))
print(pairs_dupl_list)

# данный формат записи пар принят для упрощения работы с ними

40017
[('489490', 3), ('489515', 3), ('489523', 3), ('489552', 3), ('489564', 3), ('489566', 3), ('489670', 3), ('489764', 3), ('489776', 3), ('489809', 3), ('489907', 3), ('489908', 3), ('489909', 3), ('4891102', 3), ('4891103', 3), ('4891115', 3), ('4891116', 3), ('4891156', 3), ('490515', 3), ('490523', 3), ('490552', 3), ('490564', 3), ('490566', 3), ('490670', 3), ('490764', 3), ('490776', 3), ('490809', 3), ('490907', 3), ('490908', 3), ('490909', 3), ('4901102', 3), ('4901103', 3), ('4901115', 3), ('4901116', 3), ('4901156', 3), ('515523', 3), ('515552', 3), ('515564', 3), ('515566', 3), ('515670', 3), ('515764', 3), ('515776', 3), ('515809', 3), ('515907', 3), ('515908', 3), ('515909', 3), ('5151102', 3), ('5151103', 3), ('5151115', 3), ('5151116', 3), ('5151156', 3), ('523552', 3), ('523564', 3), ('523566', 3), ('523670', 3), ('523764', 3), ('523776', 3), ('523809', 3), ('523907', 3), ('523908', 3), ('523909', 3), ('5231102', 3), ('5231103', 3), ('5231115', 3), ('5231116', 3),

In [7]:
# ШАГ 4

# Расчитаем число повторений (частотность) пар курсов в покупках пользователей
counter_pairs = Counter(pairs_list)
print(counter_pairs.most_common())
print(len(counter_pairs))
counter_pairs_dupl = Counter(pairs_dupl_list)
print(counter_pairs_dupl.most_common())
print(len(counter_pairs_dupl))
counter_pairs_with_dupl = counter_pairs + counter_pairs_dupl
print(len(counter_pairs_with_dupl), counter_pairs_with_dupl)

[(('551566', 3), 797), (('515551', 3), 417), (('489551', 3), 311), (('523551', 3), 304), (('566794', 3), 290), (('489515', 3), 286), (('490566', 3), 253), (('490551', 3), 247), (('570752', 3), 247), (('569572', 3), 216), (('515523', 3), 213), (('553745', 3), 212), (('489523', 3), 206), (('569840', 3), 204), (('514551', 3), 200), (('516745', 3), 199), (('515566', 3), 195), (('489566', 3), 188), (('504572', 3), 184), (('572840', 3), 178), (('551552', 3), 177), (('507570', 3), 172), (('490809', 3), 163), (('489490', 3), 152), (('507752', 3), 150), (('523552', 3), 144), (('490515', 3), 143), (('551570', 3), 142), (('514515', 3), 139), (('504569', 3), 139), (('514566', 3), 138), (('551745', 3), 138), (('502551', 3), 135), (('504840', 3), 135), (('5711125', 3), 122), (('523566', 3), 120), (('502566', 3), 120), (('570809', 3), 119), (('752809', 3), 115), (('490523', 3), 114), (('357571', 3), 112), (('523564', 3), 110), (('551749', 3), 109), (('551777', 3), 107), (('516553', 3), 107), (('55167

In [8]:
# ШАГ 5

# Рассчитаем определенные Правилами построения рекомендательной системы границы
# отсечения курсов (5%, 10%), нерекомендуемых к покупке исходя из частоты их
# повторения в парах с другими курсами
max_rep = counter_pairs_with_dupl.most_common(1)
max_rep = dict(max_rep).values()
max_rep = list(max_rep)
max_rep = max_rep[0]
print('максимальное число повторений пары курсов - ',max_rep)
border_rep_1 = round(max_rep * .05)
border_rep_2 = round(max_rep * .1)
print('5%-е число повторений от максиального числа повторений - ',border_rep_1)
print('10%-е число повторений от максиального числа повторений - ',border_rep_2)

максимальное число повторений пары курсов -  797
5%-е число повторений от максиального числа повторений -  40
10%-е число повторений от максиального числа повторений -  80


In [9]:
# ШАГ 6

# Получим таблицу пар, число повторений которых не меньше 5% от числа повторений
# самой/ых часто встречающейся/ихся пары/пар (эта таблица станет основой для
# промежуточной таблицы рекомендаций, которые будут фомироваться исходя из частоты
# повторения курсов в парах с другими курсами)
pairs_sort = counter_pairs_with_dupl.most_common()
pair_rating_dict = dict(pairs_sort)

col_1 = pair_rating_dict.keys()
col_2 = pair_rating_dict.values()

recommend_base = pd.DataFrame({'ind': col_1, 'count': col_2})
recommend_base['sep_bord'] = recommend_base['count'].\
            apply(lambda x: 'True' if x > border_rep_1 else 'False')
recommend_base['purch_part_of_pair'] = recommend_base['ind'].\
            apply(lambda x: x[0][:x[1]])
recommend_base['recomm_part_of_pair'] = recommend_base['ind'].\
            apply(lambda x: x[0][x[1]:])

recommend_base_prep = recommend_base[recommend_base['sep_bord'] == 'True']
recommend_base_prep = recommend_base_prep.drop('sep_bord', axis=1)

print(recommend_base_prep)


# Найдем число повторений "первого" курса в паре, того курса (purchased_part_of_pair),
# для которого будем подбирать рекомендуемый курс (recomm_part_of_pair), в двух (если
# такие имеются) самых часто встречающихся с этим курсом парах с учетом границы отсечения
# в 10-ть %-ов
def pair_prep (crs):
    mask = recommend_base_prep['purch_part_of_pair'] == crs
    df = recommend_base_prep[mask].sort_values('count', ascending = False)
    set_rep = list()
    rec1 = df.iloc[0, 1]
    set_rep.append(rec1)
    if (len(df) >= 2) and (df.iloc[1, 1] > border_rep_2):
        rec2 = df.iloc[1, 1]
        set_rep.append(rec2)
    return set_rep
    
recommend_base_prep['repet_purch_part'] = \
        recommend_base_prep['purch_part_of_pair'].apply(pair_prep)

#print(recommend_base_prep)


# Присвоим парам номера для подбора рекомендаций. 
# num_recom 1 - пара для рекомендации №1, # num_recom 2 - пара для рекомендации №2.
# Пары, которые не подходят для рекомендаций, отсеиваются. 
def num_recom (row):
    num = ''
    rep = recommend_base_prep.iloc[row,4]
    len_rep = len(rep)
    if rep[0] == recommend_base_prep.iloc[row,1]: 
        num = '1'
    elif (len_rep == 2) and (rep[1] == recommend_base_prep.iloc[row,1]):
        num = '2'
    else:
        num = 'Nan'
    return num   
     
recommend_base_prep['index'] = recommend_base_prep.index  
recommend_base_prep['num_recom'] = recommend_base_prep['index'].apply(num_recom)
recommend_base_prep = recommend_base_prep[recommend_base_prep['num_recom'] != 'Nan'] 
recommend_base_prep = recommend_base_prep.drop('index', axis=1)
recommend_base_prep = recommend_base_prep.drop('repet_purch_part', axis=1)
recommend_base_prep = recommend_base_prep.sort_values(by=['purch_part_of_pair',\
    'num_recom'], ignore_index=True)

print(recommend_base_prep)

              ind  count purch_part_of_pair recomm_part_of_pair
0     (551566, 3)    797                551                 566
1     (566551, 3)    797                566                 551
2     (515551, 3)    417                515                 551
3     (551515, 3)    417                551                 515
4     (489551, 3)    311                489                 551
..            ...    ...                ...                 ...
351  (5071115, 3)     41                507                1115
352   (562551, 3)     41                562                 551
353   (777679, 3)     41                777                 679
354   (572566, 3)     41                572                 566
355  (1115507, 4)     41               1115                 507

[356 rows x 4 columns]
             ind  count purch_part_of_pair recomm_part_of_pair num_recom
0   (1099568, 4)     53               1099                 568         1
1   (1099745, 4)     53               1099                 745

In [10]:
recommend_base_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ind                  88 non-null     object
 1   count                88 non-null     int64 
 2   purch_part_of_pair   88 non-null     object
 3   recomm_part_of_pair  88 non-null     object
 4   num_recom            88 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.6+ KB


In [11]:
# ШАГ 7

# Найдем два самых востребованных курса для формирования рекомендаций в случаях,
# когда курс в парах с другими курсами пользователями не покупался или покупался
# слишком редко, т.е. в случаях, когда предыдущая таблица recommend_base_prep не
# содержит нужной рекомендации
resource_id_series = Purch_Courses_On_User_df['resource_id']
resource_id_counter = Counter(resource_id_series)

resource_id_most_common = resource_id_counter.most_common(2)
course_recom_1 = str(resource_id_most_common[0][0])
course_recom_2 = str(resource_id_most_common[1][0])
print('курс 1 - ', course_recom_1)
print('курс 2 - ', course_recom_2)

курс 1 -  551
курс 2 -  566


In [12]:
# ШАГ 8

# Определим полный список курсов, для которых строим систему рекомендаций
resource_id_list = list(resource_id_counter)
print(resource_id_list)

[516, 1099, 356, 1125, 1187, 745, 679, 357, 571, 553, 1147, 361, 1138, 810, 1115, 562, 1140, 566, 568, 794, 551, 833, 514, 517, 672, 363, 511, 563, 509, 1144, 1188, 552, 513, 1141, 750, 490, 840, 508, 744, 862, 1139, 908, 800, 569, 1104, 765, 503, 749, 1100, 1103, 502, 564, 865, 764, 523, 1186, 752, 366, 367, 519, 504, 809, 515, 912, 489, 864, 741, 1101, 1146, 776, 803, 671, 753, 829, 1198, 1102, 659, 909, 777, 518, 907, 360, 813, 835, 814, 1116, 1161, 863, 572, 743, 1124, 1128, 742, 570, 507, 664, 1185, 791, 365, 1156, 756, 359, 755, 362, 358, 1184, 368, 911, 1160, 757, 1181, 1145, 1152, 364, 866, 1201, 834, 670, 1129, 1199, 836, 1182, 902, 837, 1200, 830]


In [13]:
# ШАГ 9 

# Составим итоговую таблицу рекомендаций
col = resource_id_list

df_recommend = pd.DataFrame({'resource_id': col})

def select_recom1(course):
    mask = recommend_base_prep['purch_part_of_pair'] == str(course)
    df_num = recommend_base_prep[mask]
    if len(df_num) != 0:
        return (df_num.iloc[0,3])
    else:
        return course_recom_1

def select_recom2(course):
    mask = recommend_base_prep['purch_part_of_pair'] == str(course)
    df_num = recommend_base_prep[mask]
    if len(df_num) == 2:
        return (df_num.iloc[1,3])
    elif (len(df_num) == 1) and (df_num.iloc[0,3] != course_recom_1):
        return course_recom_1
    else:
        return course_recom_2
   
df_recommend['resource_recommend_1'] = df_recommend['resource_id'].apply(select_recom1)
df_recommend['resource_recommend_2'] = df_recommend['resource_id'].apply(select_recom2)
df_recommend = df_recommend.sort_values(by='resource_id', ignore_index=True)

print(df_recommend)

     resource_id resource_recommend_1 resource_recommend_2
0            356                  571                  357
1            357                  571                  356
2            358                  551                  566
3            359                  551                  566
4            360                  551                  566
..           ...                  ...                  ...
121         1188                  551                  566
122         1198                  551                  566
123         1199                  551                  566
124         1200                  551                  566
125         1201                  551                  566

[126 rows x 3 columns]


In [14]:
df_recommend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   resource_id           126 non-null    int64 
 1   resource_recommend_1  126 non-null    object
 2   resource_recommend_2  126 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.1+ KB


In [15]:
#df_recommend.to_csv("df_recommend.csv", index=False, sep=",")

##### Вывод: 
Мы получили Таблицу рекомендаций курсов (df_recommend), которую планируем использовать при оформлении пользователями корзин покупок, предлагая им курсы, которые могут быть им интересны, и тем самым повысить средний чек. Мы видим, что в полученной Таблице меньше половины рекомендаций сделаны на основе личных предпочтений. Это вытекает из следующих расчетов: Таблица recommend_base_prep содержит только 88 комбинаций пар, которые будут использоваться для формирования рекомендаций на основе личных предпочтений, т.е. только в 88 случаях из 252 (126 х 2 = 252 варианта), все остальные рекомендации (252 - 88 = 164 варианта) будут даваться на основе условно общепользовательских предпочтений (т.е. будут рекомендоваться два самых продаваемых за 2017-2018 гг. курса).
Кроме того, мы видим, что статистика, которую мы можем использовать для формирования личных предпочтений довольно мала. Из 70424 покупок мы получили 3989 пар курсов и только 356 из них, при наложении довольно низкой (т.е. накладывающей мягкое ограничение) границы отсечения по частотности (5% от максимального числа повторений пары/пар курсов), "могли претендовать" на использование их в рекомендательной системе, а после исключения ненужных (3-х, 4-х и т.д.) дубликатов и наложения второй границы отсечения (в 10-ть %-в) осталось только 88 пар курсов из 252 необходимых (по две рекомендации на каждый приобретенный в 2017-2018 гг. курс, по которым, соответственно, за эти годы была накоплена статистика, ставшая основой для разработки данной рекомендательной системы). При этом самая часто встречающаяся пара (551-566) повторялась 797 раз, следующая по частотности пара (515-551) - 417 раз, т.е. из 12656 пользователей, купивших более одного курса, только 797 и 417 пользователей, соответственно, приобрели курсы в указанных комбинациях (551-566, 515-551). Что, возможно, для указанных продуктов не так уж и мало, но при составлении прогнозов (т.е. формировании рекомендаций) может дать существенную погрешность. 
При этом нельзя забывать о том, что в случаях, когда для формирования рекомендаций используются условно общепользовательские предпочтения, пользователям, по установленным ранее правилам, предлагаются два самых продаваемых курса, что, с одной стороны, призвано увеличивать средний чек, т.к. пользователя с большей вероятностью заинтересуют рекомендации "из популярного", а с другой стороны, акцентирует внимание на курсах, которые и без того неплохо продаются. Возможно, на одной из тестируемых групп стоит опробовать вариант, при котором за условно общепользовательские предпочтения будут приняты курсы, следующие по популярности на 3-м и 4-м местах после двух самых продаваемых.